In [ ]:
# Importing modules
import torch as to
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch.utils.data as to_data
from torch.utils.tensorboard import SummaryWriter as sumwriter

In [ ]:
# Specify hardware for ML training (GPU default)
device = "cuda" if to.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Quickly generate list of strings for frequency numbers and ratios
def freq_name(no_freq, include_freq=True, include_ratio=True):
    """
    Creates an ordered list of string from inputted parameters:

    no_freq = (int) number of desired frequencies
    include_freq = (bool) include the individual frequencies or not (default True)
    include_ratio = (bool) include the non-trivial ratios between frequencies or not (default True)
    """
    names = []
    if include_freq:
        for i in range(no_freq):
            names.append('f'+str(i+1))
    if include_ratio:
        for i in range(no_freq):
            for j in range(i):
                names.append('f'+str(i+1)+'/f'+str(j+1))
    return names

In [ ]:
# Create Pytorch dataset class for data batching during training
class SAE_data(to_data.Dataset):
    def __init__(self, scaled_dataframe, X_names, Y_names):
        self.len = len(scaled_dataframe)
        self.X = to.from_numpy(scaled_dataframe[X_names].to_numpy().astype('float32')).to(device)
        self.Y = to.from_numpy(scaled_dataframe[Y_names].to_numpy().astype('float32')).to(device)

    def __len__(self):
        return self.len
  
    def __getitem__(self, idx):
        X_idx = self.X[idx,:]
        Y_idx = self.Y[idx,:]
        return X_idx, Y_idx

In [ ]:
def activation(activ_name):
    if activ_name=='relu':
        return to.nn.ReLU()
    elif activ_name=='lrelu':
        return to.nn.LeakyReLU()
    elif activ_name=='prelu':
        return to.nn.PReLU()
    elif activ_name=='relu6':
        return to.nn.ReLU6()
    elif activ_name=='sigmoid':
        return to.nn.Sigmoid()
    elif activ_name=='tanh':
        return to.nn.Tanh()
    elif activ_name=='silu':
        return to.nn.SiLU()
    elif activ_name=='selu':
        return to.nn.SELU()
    elif activ_name=='celu':
        return to.nn.CELU()
    elif activ_name=='gelu':
        return to.nn.GELU()
    else:
        return to.nn.ReLU()

In [ ]:
class SAE_Network(to.nn.Module):
    def __init__(self, num_X, num_Y, he_nodes, hd_nodes, hactiv_type):
        super(SAE_Network, self).__init__()

        self.encoder = []
        self.encoder.append(to.nn.Linear(num_X, he_nodes[0]))
        self.encoder.append(activation(hactiv_type))

        for i in range(len(he_nodes)-1):
            self.encoder.append(to.nn.Linear(he_nodes[i], he_nodes[i+1]))
            self.encoder.append(activation(hactiv_type))

        self.encoder.append(to.nn.Linear(he_nodes[-1], num_Y))

        self.encoder = to.nn.Sequential(*self.encoder).to(device)
        for i in self.encoder[::2]:
            to.nn.init.xavier_uniform_(i.weight)
            to.nn.init.zeros_(i.bias)

        self.decoder = []
        self.decoder.append(to.nn.Linear(num_Y, hd_nodes[0]))
        self.decoder.append(activation(hactiv_type))

        for i in range(len(hd_nodes)-1):
            self.decoder.append(to.nn.Linear(hd_nodes[i], hd_nodes[i+1]))
            self.decoder.append(activation(hactiv_type))

        self.decoder.append(to.nn.Linear(hd_nodes[-1], num_X))
        
        self.decoder = to.nn.Sequential(*self.decoder).to(device)
        for i in self.decoder[::2]:
            to.nn.init.xavier_uniform_(i.weight)
            to.nn.init.zeros_(i.bias)

    def forward(self, X):
        Y = self.encoder(X)
        Xpred = self.decoder(Y)
        return Y, Xpred

In [ ]:
def train_epoch(
    network, num_freq,
    train_dataloader,
    loss_function, optimizer,
    tb_writer, epoch_ind
    ):

    loss_list = []
    loss_listY = []
    loss_listX = []

    MAPE_listY = []
    MAPE_listX = []

    for i, data in enumerate(train_dataloader):
        X, Y = data

        if epoch_ind==0 and i==0:
            tb_writer.add_graph(network, X, verbose=False)

        optimizer.zero_grad()
        predictY, predictX = network(X)

        lossY = loss_function(predictY, Y)
        lossX = loss_function(predictX, X)
        loss = lossY + lossX

        loss_listY.append(lossY.item())
        loss_listX.append(lossX.item())
        loss_list.append(loss.item())

        loss.backward()
        optimizer.step()

        Y[:,0:num_freq] = to.exp(Y[:,0:num_freq])
        predictY[:,0:num_freq] = to.exp(predictY[:,0:num_freq])
 
        MAPEY = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPEX = to.mean(to.abs((X - predictX) / X)*100)

        MAPE_listY.append(MAPEY.item())
        MAPE_listX.append(MAPEX.item())
    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_lossY = to.mean(to.tensor(loss_listY, device=device)).item()
    mean_lossX = to.mean(to.tensor(loss_listX, device=device)).item()

    mean_MAPEY = to.mean(to.tensor(MAPE_listY, device=device)).item()
    mean_MAPEX = to.mean(to.tensor(MAPE_listX, device=device)).item()

    return mean_loss, mean_lossY, mean_lossX, mean_MAPEY, mean_MAPEX

def valid_epoch(
    network, num_freq,
    valid_dataloader,
    loss_function
    ):

    loss_list = []
    loss_listY = []
    loss_listX = []

    MAPE_listY = []
    MAPE_listX = []

    for i, data in enumerate(valid_dataloader):
        X, Y = data
        predictY, predictX = network(X)

        lossY = loss_function(predictY, Y)
        lossX = loss_function(predictX, X)
        loss = lossY + lossX

        loss_listY.append(lossY.item())
        loss_listX.append(lossX.item())
        loss_list.append(loss.item())
        
        Y[:,0:num_freq] = to.exp(Y[:,0:num_freq])
        predictY[:,0:num_freq] = to.exp(predictY[:,0:num_freq])
 
        MAPEY = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPEX = to.mean(to.abs((X - predictX) / X)*100)

        MAPE_listY.append(MAPEY.item())
        MAPE_listX.append(MAPEX.item())
    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_lossY = to.mean(to.tensor(loss_listY, device=device)).item()
    mean_lossX = to.mean(to.tensor(loss_listX, device=device)).item()

    mean_MAPEY = to.mean(to.tensor(MAPE_listY, device=device)).item()
    mean_MAPEX = to.mean(to.tensor(MAPE_listX, device=device)).item()

    return mean_loss, mean_lossY, mean_lossX, mean_MAPEY, mean_MAPEX

In [ ]:
def train_AEI(
    network, num_freq,
    train_dataloader, valid_dataloader,
    loss_function, optimizer_type,
    epochs, learn_rate
    ):

    if optimizer_type=='adam':
        optimizer = to.optim.Adam(network.parameters(), lr=learn_rate)
    else:
        optimizer = to.optim.SGD(network.parameters(), lr=learn_rate)
    
    tb_writer = sumwriter()
    
    for i in range(epochs):
        network.train(True)
        mloss, mlossY, mlossX, mMAPEY, mMAPEX = train_epoch(network, num_freq, train_dataloader, loss_function, optimizer, tb_writer, i)

        network.eval()
        with to.no_grad():
            vmloss, vmlossY, vmlossX, vmMAPEY, vmMAPEX = valid_epoch(network, num_freq, valid_dataloader, loss_function)
        

        print('-'*50)
        print('Epoch {} / {}'.format(i+1,epochs))
        print('-'*15)
        print('Average Train Loss : {}'.format(mloss))
        print('Average Validation Loss : {}'.format(vmloss))

        tb_writer.add_scalars("Batch Mean Loss",
                            {
                                'Train' : mloss,
                                'Validation' : vmloss
                            }, i+1)

        tb_writer.add_scalars("Batch MAPE - Latent Space",
                            {
                                'Train' : mMAPEY,
                                'Validation' : vmMAPEY
                            }, i+1)

        tb_writer.add_scalars("Batch MAPE - Reconstruction",
                            {
                                'Train' : mMAPEX,
                                'Validation' : vmMAPEX
                            }, i+1)

        tb_writer.add_scalars("Batch Mean Loss - Latent Space",
                            {
                                'Train' : mlossY,
                                'Validation' : vmlossY
                            }, i+1)
    
        tb_writer.add_scalars("Batch Mean Loss - Reconstruction",
                            {
                                'Train' : mlossX,
                                'Validation' : vmlossX
                            }, i+1)
                            
    tb_writer.flush()
    tb_writer.close()

In [ ]:
# Prepare Data
data = pd.read_csv('TrainValid_Data.csv')
data = data.iloc[np.random.choice(50000,5000,False)]
num_freq = 4

features = ['E', 'nu', 'rho', 'a', 'b', 't']
latent = freq_name(num_freq,1,0)

train_split = int(0.8*len(data))
valid_split = len(data)- train_split

scaled_data = data[features+latent].copy()

scaled_data['E'] /= 1e11
scaled_data['rho'] /= 1e4
scaled_data['t'] *= 100
scaled_data[freq_name(num_freq,1,0)] = np.log(scaled_data[freq_name(num_freq,1,0)])

scaled_data = SAE_data(scaled_data, features, latent)
train_set, valid_set = to_data.random_split(scaled_data, [train_split, valid_split])

In [ ]:
# Train Model
# Parameters
num_features = len(features)
num_latent = len(latent)
he_nodes = [20,40,80,160,80,40,20]
hd_nodes = [20,40,80,160,80,40,20]
hactiv = 'silu'

batch_size_train = 50
batch_size_valid = 2000

epochs = 400
learn_rate = 1e-3

# Optim Selections
loss_function = to.nn.SmoothL1Loss()
optimizer_type = 'adam'

# Data loaders
train_loader = to.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=True)
valid_loader = to.utils.data.DataLoader(valid_set, batch_size=batch_size_valid, shuffle=True)

# Model
model = SAE_Network(num_features, num_latent, he_nodes, hd_nodes, hactiv)

In [ ]:
# Train
train_AEI(
    model, num_freq,
    train_loader, valid_loader,
    loss_function, optimizer_type,
    epochs, learn_rate)

to.save(model.state_dict(), 'SAErev_model.state')

In [ ]:
model.load_state_dict(to.load('SAErev_model.state'))

test_data = pd.read_csv('TrainValid_Data.csv')

train_split = int(0.8*len(data))
valid_split = len(data)- train_split

scaled_test_data = test_data[features+latent].copy()

scaled_test_data['E'] /= 1e11
scaled_test_data['rho'] /= 1e4
scaled_test_data['t'] *= 100
scaled_test_data[freq_name(num_freq,1,0)] = np.log(scaled_test_data[freq_name(num_freq,1,0)])

scaled_test_data = SAE_data(scaled_test_data, features, latent)
test_loader = to.utils.data.DataLoader(scaled_test_data, batch_size=len(scaled_test_data), shuffle=False)

model.eval()
with to.no_grad():
    for i, data in enumerate(test_loader):
        X, Y = data
        predictX = model.decoder(Y)
        abs_perc_error = to.abs((X - predictX)/X)*100
        MAPE_per_dim = to.mean(abs_perc_error, 0)

        print('Absolute Percentage Errors: ')
        print('-'*20)
        print('E : {:0.2f} %'.format(MAPE_per_dim[0].item()))
        print('nu : {:0.2f} %'.format(MAPE_per_dim[1].item()))
        print('rho : {:0.2f} %'.format(MAPE_per_dim[2].item()))
        print('a : {:0.2f} %'.format(MAPE_per_dim[3].item()))
        print('b : {:0.2f} %'.format(MAPE_per_dim[4].item()))
        print('t : {:0.2f} %'.format(MAPE_per_dim[5].item()))

        plt.figure(figsize=(10,10))
        plt.plot(['E', r'$\nu$', r'$\rho$', 'a', 'b', 't'], MAPE_per_dim.cpu(), 'b:o')
        plt.ylim(0,100)
        plt.grid(axis='y')
        plt.yticks(np.arange(0,105,5))
        plt.xticks(fontsize=16)
        plt.title('SAErev MAPE Plot', fontsize=20)
        plt.ylabel('MAPE [%]', fontsize=18)
        plt.xlabel('Parameter', fontsize=18)
        plt.savefig('SAErev_MAPE_plot.pdf', dpi=1200, bbox_inches='tight')
        plt.show()

        np.savetxt('SAErev_MAPE_test.txt', MAPE_per_dim.cpu().numpy())